In [1]:
import QuantLib as ql

In [32]:
# we define a CommonVars class for creating all variables or arguments that we need in below case.
class CommonVars:

    def __init__(self):
        self.nominals = []
        self.nominals.append(100.0)
        self.frequency = ql.Quarterly
        self.termStructure = ql.RelinkableYieldTermStructureHandle()
        self.index = ql.Euribor3M(self.termStructure)
        self.calendar = self.index.fixingCalendar()
        self.businessConvention = ql.ModifiedFollowing
        today = ql.Date(23,5,2023)
        ql.Settings.instance().evaluationDate = today
        self.fixingDays = 2
        self.settlement = self.calendar.advance(today, self.fixingDays, ql.Days)
        self.termStructure.linkTo(ql.FlatForward(self.settlement, 0.05, ql.Actual365Fixed()))
    
    def makeLeg(self, startDate, length, timeUnit):
        endDate = self.calendar.advance(startDate, length, timeUnit, self.businessConvention)
        legSchedule = ql.Schedule(startDate, endDate, ql.Period(self.frequency),
                               self.calendar, self.businessConvention, self.businessConvention,
                               ql.DateGeneration.Forward, False)
        leg = ql.IborLeg(self.nominals,legSchedule, self.index, ql.Actual365Fixed(), 
                         self.businessConvention, 
                         fixingDays=[2], gearings=[1])
        return leg
    
    def makeEngine(self, volatility):
        return ql.BlackCapFloorEngine(self.termStructure, ql.QuoteHandle(ql.SimpleQuote(volatility)))
    
    def makeCapFloor(self, capfloorType, leg, strike, volatility):
        strikes = []
        strikes.append(strike)
        if capfloorType == "Cap":
            result = ql.Cap(leg, strikes)
            result.setPricingEngine(self.makeEngine(volatility))
            return result
        elif capfloorType == "Floor":
            result = ql.Floor(leg, strikes)
            result.setPricingEngine(self.makeEngine(volatility))
            return result
        else:
            return "unknown cap/floor type"
        


In [74]:
#setup CommonVars object
vars = CommonVars()
print(f'vars: {vars.settlement}')

#testing mkt data
vol = 0.2
strike = 0.04

# initial cap/floor product
leg = vars.makeLeg(vars.settlement, 12, ql.Months)
cap = vars.makeCapFloor("Cap", leg, strike, vol)
cap.setPricingEngine(vars.makeEngine(vol))
print(f"Cap pricing result, npv: {cap.NPV()}")

vars: May 25th, 2023
Cap pricing result, npv: 0.9549248835849425


In [75]:
import pandas as pd

leg = cap.floatingLeg()

dates=[]
amt=[]
for cf in leg:
    dates.append(cf.date().ISO())
    amt.append(cf.amount())

caplets = pd.DataFrame({
    'dates': dates,
    'amounts': amt,
    'atmForward': cap.optionletsAtmForward(),
    'price' :cap.optionletsPrice()
})

display(caplets)

,dates,amounts,atmForward,price
0,2023-08-25,1.250876,0.049627,0.239617
1,2023-11-27,1.278244,0.049634,0.242498
2,2024-02-26,1.237194,0.049624,0.235261
3,2024-05-27,1.237194,0.049624,0.237548


In [76]:
# using Hull-White tree model
# calibrate HW paramters
# define model
modelHW = ql.HullWhite(vars.termStructure)
# define calibration helper
index = vars.index
quote = ql.SimpleQuote(vol)
capHelper = ql.CapHelper(ql.Period(5, ql.Years), ql.QuoteHandle(quote),
                         index, ql.Semiannual, ql.Actual365Fixed(), True,
                         vars.termStructure, ql.BlackCalibrationHelper.PriceError)

helperList = []
helperList.append(capHelper)
for h in helperList:
    h.setPricingEngine(ql.AnalyticCapFloorEngine(modelHW, vars.termStructure))

# do model calibration
optimize = ql.BFGS()
modelHW.calibrate(helperList, optimize, ql.EndCriteria(200, 100, 0.00000001, 0.00000001, 0.00000001))
for h in helperList:
    npv = h.modelValue()
    impliedVol = h.impliedVolatility(npv, 0.0001, 1000, 0.05, 0.8)
    print(f"model value is : {npv}")
    print(f"implied vol is : {impliedVol}")

print(f"HW model mean-reversion : {modelHW.params()[0]}")
print(f"HW model volatility (sigma) : {modelHW.params()[1]}")

# CapFloor Tree pricing Engine
hwTreeEngine = ql.TreeCapFloorEngine(modelHW, 40, vars.termStructure)
cap.setPricingEngine(hwTreeEngine)
print("cap pricing result with HW tree model")
print(f"cap NPV : {cap.NPV()}")

# Monte Carlo simulation
# mcHWEngine = ql.MCHullWhiteCapFloorEngine <-- the class did not implemented in Python

model value is : 0.02267504274081755
implied vol is : 0.2
HW model mean-reversion : 0.09998497040729101
HW model volatility (sigma) : 0.011575998999501758
cap pricing result with HW tree model
cap NPV : 1.0362703563577056
